# Visualize Results: Downstream Performance - Regression Corrupted Experiments

[Set Average Best Imputation Method Manually](#Set-Average-Best-Imputation-Method-Manually)

This notebook should answer the questions: *Does imputation lead to better downstream performances?*

Data needs to be preprocessed with other notebook, her we only import two csv files with raw data regarding the results of the experiment and information about the used datasets!


In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import seaborn as sns
from pandas.api.types import CategoricalDtype
from pathlib import Path

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr


%matplotlib inline

%load_ext autoreload
%autoreload 2

## Settings

In [2]:
sns.set(style="whitegrid")
sns.set_context('paper', font_scale=1.5)
mpl.rcParams['lines.linewidth'] = '2'

In [3]:
CLF_METRIC = "Classification Tasks"
REG_METRIC = "Regression Tasks"

DOWNSTREAM_RESULT_TYPE = "downstream_performance_mean"
IMPUTE_RESULT_TYPE = "impute_performance_mean"


## Data Preparation

In [4]:
# import preprocessed data from experiments
results = pd.read_csv('../multicolumn_corrupted.csv')
results

,experiment,imputer,task,missing_type,missing_fraction,strategy,column,result_type,metric,train,test,baseline,corrupted,imputed
0,experiment_multi_column_corrupted,KNNImputer,30,MNAR,0.50,single_all,eccen,impute_performance_std,MAE,10.103773,0.295436,NaN,NaN,NaN
1,experiment_multi_column_corrupted,KNNImputer,30,MNAR,0.50,single_all,eccen,impute_performance_std,MSE,223.302351,0.301545,NaN,NaN,NaN
2,experiment_multi_column_corrupted,KNNImputer,30,MNAR,0.50,single_all,eccen,impute_performance_std,RMSE,10.103773,0.295436,NaN,NaN,NaN
3,experiment_multi_column_corrupted,KNNImputer,30,MNAR,0.30,single_all,eccen,impute_performance_std,MAE,10.103773,0.295436,NaN,NaN,NaN
4,experiment_multi_column_corrupted,KNNImputer,30,MNAR,0.30,single_all,eccen,impute_performance_std,MSE,223.302351,0.301545,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24787,experiment_multi_column_corrupted,VAEImputer,310,MCAR,0.01,single_all,attr2,downstream_performance_mean,F1_macro,NaN,NaN,0.774143,0.0,0.774143
24788,experiment_multi_column_corrupted,VAEImputer,310,MCAR,0.01,single_all,attr2,downstream_performance_mean,F1_weighted,NaN,NaN,0.982959,0.0,0.982959
24789,experiment_multi_column_corrupted,VAEImputer,310,MCAR,0.10,single_all,attr2,downstream_performance_mean,F1_micro,NaN,NaN,0.984801,0.0,0.985248
24790,experiment_multi_column_corrupted,VAEImputer,310,MCAR,0.10,single_all,attr2,downstream_performance_mean,F1_macro,NaN,NaN,0.760027,0.0,0.770133


In [5]:
# Filtering the relevant data for downstream analysis

na_impute_results = results[
    (results["result_type"] == IMPUTE_RESULT_TYPE) & 
    (results["metric"].isin(["F1_macro", "RMSE"]))
]
na_impute_results.drop(["baseline", "corrupted", "imputed"], axis=1, inplace=True)
na_impute_results = na_impute_results[na_impute_results.isna().any(axis=1)]
na_impute_results.shape


/tmp/ipykernel_2568289/2463847502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  na_impute_results.drop(["baseline", "corrupted", "imputed"], axis=1, inplace=True)


(0, 11)

In [6]:
# check if strategy type is correct!
STRATEGY_TYPE = "single_all"

downstream_results = results[
    (results["result_type"] == DOWNSTREAM_RESULT_TYPE) & 
    (results["metric"].isin(["F1_macro", "RMSE"]) &
    (results["strategy"] == STRATEGY_TYPE))
]

# remove experiments where imputation failed
downstream_results = downstream_results.merge(
    na_impute_results,
    how = "left",
    validate = "one_to_one",
    indicator = True,
    suffixes=("", "_imp"),
    on = ["experiment", "imputer", "task", "missing_type", "missing_fraction", "strategy", "column"]
)
downstream_results = downstream_results[downstream_results["_merge"]=="left_only"]

assert len(results["strategy"].unique()) == 1
downstream_results.drop(["experiment", "strategy", "result_type_imp", "metric_imp", "train", "test", "train_imp", "test_imp", "_merge"], axis=1, inplace=True)

downstream_results = downstream_results.rename(
    {
        "imputer": "Imputation_Method",
        "task": "Task",
        "missing_type": "Missing Type",
        "missing_fraction": "Missing Fraction",
        "column": "Column",
        "baseline": "Baseline",
        "imputed": "Imputed",
        "corrupted": "Corrupted"
    },
    axis = 1
)

In [7]:
rename_imputer_dict = {
    "ModeImputer": "Mean/Mode",
    "KNNImputer": "KNN",
    "ForestImputer": "Random Forest",
    "AutoKerasImputer": "Discriminative DL",
    "VAEImputer": "VAE",
    "GAINImputer": "GAIN"    
}

rename_metric_dict = {
    "F1_macro": CLF_METRIC,
    "RMSE": REG_METRIC
}

downstream_results = downstream_results.replace(rename_imputer_dict)
downstream_results = downstream_results.replace(rename_metric_dict)

downstream_results

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed
0,KNN,30,MNAR,0.50,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
1,KNN,30,MNAR,0.30,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
2,KNN,30,MNAR,0.01,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
3,KNN,30,MNAR,0.10,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
4,KNN,30,MAR,0.50,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
...,...,...,...,...,...,...,...,...,...,...
2061,VAE,310,MAR,0.10,attr2,downstream_performance_mean,Classification Tasks,0.780147,0.0,0.780147
2062,VAE,310,MCAR,0.50,attr2,downstream_performance_mean,Classification Tasks,0.761074,0.0,0.744672
2063,VAE,310,MCAR,0.30,attr2,downstream_performance_mean,Classification Tasks,0.762688,0.0,0.770352
2064,VAE,310,MCAR,0.01,attr2,downstream_performance_mean,Classification Tasks,0.774143,0.0,0.774143


### Robustness: check which imputers yielded `NaN`values

In [8]:
for col in downstream_results.columns:
    na_sum = downstream_results[col].isna().sum()
    if na_sum > 0:
        print("-----" * 10)        
        print(col, na_sum)
        print("-----" * 10)        
        na_idx = downstream_results[col].isna()
        print(downstream_results.loc[na_idx, "Imputation Method"].value_counts(dropna=False))
        print("\n")

## Adding Dataset Info, Sorting and Ranking

In [9]:
clf_row_idx = downstream_results["metric"] == CLF_METRIC
reg_row_idx = downstream_results["metric"] == REG_METRIC

# Sorting of data

#adjust order to fit the processing time -> fastest first
methods_order = CategoricalDtype(['Mean/Mode', 'KNN', 'Random Forest', 'VAE', 'GAIN', 'Discriminative DL'], ordered=True)
downstream_results_full_sort = downstream_results.copy()

downstream_results_full_sort['Imputation_Method'] = downstream_results_full_sort['Imputation_Method'].astype(methods_order)
downstream_results_full_sort = downstream_results_full_sort.sort_values(['Task', 'Missing Type',
                                                                         'Missing Fraction', 'Imputed','Imputation_Method'], ascending=[True, True, True, True, True])


downstream_results.head()

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed
0,KNN,30,MNAR,0.50,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
1,KNN,30,MNAR,0.30,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
2,KNN,30,MNAR,0.01,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
3,KNN,30,MNAR,0.10,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862
4,KNN,30,MAR,0.50,eccen,downstream_performance_mean,Classification Tasks,0.680862,0.0,0.680862


In [10]:
#downstream_results_full_sort.info()

In [11]:
# add dataset information from other csv file
dataset_info = pd.read_csv('../../datasets_information_overview.csv')
dataset_info = dataset_info.rename(columns={"did": "Task"})


downstream_results_full_sort = pd.merge(downstream_results_full_sort, dataset_info, on='Task')
downstream_results_full_sort.head()

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses
0,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
1,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.726188,0.0,0.726121,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
2,Mean/Mode,6,MAR,0.10,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
3,KNN,6,MAR,0.10,x-box,downstream_performance_mean,Classification Tasks,0.726188,0.0,0.726121,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
4,Mean/Mode,6,MAR,0.30,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0


In [12]:
# Ranking of downstream performance per data constellation for every imputation method

EXPERIMENTAL_CONDITIONS = ["Task", "Missing Type", "Missing Fraction", "Column", "result_type"]

downstream_results_rank = downstream_results_full_sort.copy()

downstream_results_rank["Downstream Performance Rank"] = downstream_results_rank.groupby(EXPERIMENTAL_CONDITIONS).rank(ascending=True, na_option="bottom", method="first")["Imputed"]
# create csv for detailled checks
downstream_results_rank.to_csv('downstream_results_complete_overview.csv')
downstream_results_rank.head()


/tmp/ipykernel_2568289/1906142079.py:7: FutureWarning: Dropping invalid columns in DataFrameGroupBy.rank is deprecated. In a future version, a TypeError will be raised. Before calling .rank, select only columns which should be valid for the function.
  downstream_results_rank["Downstream Performance Rank"] = downstream_results_rank.groupby(EXPERIMENTAL_CONDITIONS).rank(ascending=True, na_option="bottom", method="first")["Imputed"]


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank
0,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0
1,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.726188,0.0,0.726121,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,2.0
2,Mean/Mode,6,MAR,0.10,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0
3,KNN,6,MAR,0.10,x-box,downstream_performance_mean,Classification Tasks,0.726188,0.0,0.726121,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,2.0
4,Mean/Mode,6,MAR,0.30,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0


In [13]:
# Adjust column type for Imputation_Method
downstream_results_rank['Imputation_Method'] = downstream_results_rank['Imputation_Method'].astype('object')

#downstream_results_rank.info()

In [14]:
# Merge the two columns "Missing Type" and "Missing Fraction"

downstream_results_rank['Missing Type'] = downstream_results_rank['Missing Type'].astype(str)
downstream_results_rank['Missing Fraction'] = downstream_results_rank['Missing Fraction'].astype(str)
#datatype_new = downstream_results_rank.dtypes

downstream_results_rank['Data_Constellation'] = downstream_results_rank['Missing Type'] + ' - ' + downstream_results_rank['Missing Fraction']
#downstream_results_rank.to_csv('downstream_results_rank_temp.csv')
downstream_results_rank_heatmap2 = downstream_results_rank.copy()
downstream_results_rank.head()


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation
0,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.01
1,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.726188,0.0,0.726121,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,2.0,MAR - 0.01
2,Mean/Mode,6,MAR,0.1,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.1
3,KNN,6,MAR,0.1,x-box,downstream_performance_mean,Classification Tasks,0.726188,0.0,0.726121,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,2.0,MAR - 0.1
4,Mean/Mode,6,MAR,0.3,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.3


## Analyzing Performance based on Rank per Data Constellation

In [15]:
data = downstream_results_rank.copy()

# Count amount of different Data constellations in column "Data_Constellation"
dc_unique = data.Data_Constellation.unique().size
print(dc_unique, "Data Constellations")
print("_____________________")
# Count amount of 1.0 Ranking result in column "Downstream Performance Rank" (Numbers must match)
rank_count = data['Downstream Performance Rank'].value_counts()
print(rank_count)
print("_____________________")
# Filter for 1.0 Ranking -> Overview -> save as csv
rank_1 = data.loc[data['Downstream Performance Rank'] == 1.0]
rank_1.to_csv('rank_1.csv')

print("_____________________")
# Count how often each Imputation Method is present -> most "wins"
rank_wins = rank_1['Imputation_Method'].value_counts()
print(rank_wins)
print("_____________________")
# Take initial overview and filter for each imputation method and calculate average rank
methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
for i in methods:
    df_average_rank = data.loc[data['Imputation_Method'] == i]
    len_ar = len(df_average_rank)
    print(len_ar, "Amount of results available")
    rank_pos = df_average_rank['Downstream Performance Rank'].value_counts().sort_index(ascending=True)
    print(rank_pos)
    average_rank = df_average_rank["Downstream Performance Rank"].mean()
    print("Average Rank for", i, "is", average_rank)





12 Data Constellations
_____________________
1.0    804
2.0    804
3.0    252
4.0    206
Name: Downstream Performance Rank, dtype: int64
_____________________
_____________________
Mean/Mode        462
KNN              281
VAE               43
Random Forest     18
Name: Imputation_Method, dtype: int64
_____________________
252 Amount of results available
1.0     18
2.0     24
3.0    142
4.0     68
Name: Downstream Performance Rank, dtype: int64
Average Rank for Random Forest is 3.0317460317460316
804 Amount of results available
1.0    281
2.0    446
3.0     65
4.0     12
Name: Downstream Performance Rank, dtype: int64
Average Rank for KNN is 1.7611940298507462
804 Amount of results available
1.0    462
2.0    300
3.0     35
4.0      7
Name: Downstream Performance Rank, dtype: int64
Average Rank for Mean/Mode is 1.486318407960199
206 Amount of results available
1.0     43
2.0     34
3.0     10
4.0    119
Name: Downstream Performance Rank, dtype: int64
Average Rank for VAE is 2.995145631

In [16]:
rank_1_backup = rank_1.copy()
rank_1

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation
0,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.01
2,Mean/Mode,6,MAR,0.1,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.1
4,Mean/Mode,6,MAR,0.3,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.3
6,Mean/Mode,6,MAR,0.5,x-box,downstream_performance_mean,Classification Tasks,0.725728,0.0,0.725661,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.5
8,Mean/Mode,6,MCAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.725083,0.0,0.725768,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MCAR - 0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,KNN,42712,MCAR,0.5,humidity,downstream_performance_mean,Regression Tasks,149.574896,0.0,149.533159,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MCAR - 0.5
2058,KNN,42712,MNAR,0.01,humidity,downstream_performance_mean,Regression Tasks,149.517948,0.0,149.519166,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.01
2060,KNN,42712,MNAR,0.1,humidity,downstream_performance_mean,Regression Tasks,149.517948,0.0,149.519166,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.1
2062,KNN,42712,MNAR,0.3,humidity,downstream_performance_mean,Regression Tasks,149.517948,0.0,149.519166,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.3


## Set Average Best Imputation Method Manually

In [17]:
# SET AVERAGE BEST IMPUTATION METHOD HERE, BASED ON THE PREVIOUS RESULTS
# Alternatively you can define a baseline method here, which will be used instead, depending on your analysis goals

AVERAGE_BEST_IMPUTATION_METHOD = "Random Forest"

## Differences in Performance Relative to Average Best Imputation Method

In [18]:
av_best = data.loc[data['Imputation_Method'] == AVERAGE_BEST_IMPUTATION_METHOD]
av_best['Task'] = av_best['Task'].astype(str)
av_best['Data_Constellation'] = av_best['Data_Constellation'] + ' - ' + av_best['Task']

av_best = av_best[['Imputation_Method', 'Imputed', 'Data_Constellation', 'Downstream Performance Rank']]
av_best = av_best.rename(columns={'Imputation_Method':'Imputation_Method_average', 
                               'Imputed':'Imputed_average',
                                 'Downstream Performance Rank':'Downstream Performance Rank Average'})


rank_1['Task'] = rank_1['Task'].astype(str)
rank_1['Data_Constellation'] = rank_1['Data_Constellation'] + ' - ' + rank_1['Task']
rank_1 = rank_1[['Imputation_Method', 'Imputed', 'Data_Constellation', 'Downstream Performance Rank']]
rank_1 = rank_1.rename(columns={'Imputation_Method':'Imputation_Method_best', 
                               'Imputed':'Imputed_best',
                               'Downstream Performance Rank':'Downstream Performance Rank Best'})

performance_difference = pd.merge(av_best, rank_1, on='Data_Constellation')
#performance_difference.head()

/tmp/ipykernel_2568289/2063573927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  av_best['Task'] = av_best['Task'].astype(str)
/tmp/ipykernel_2568289/2063573927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  av_best['Data_Constellation'] = av_best['Data_Constellation'] + ' - ' + av_best['Task']
/tmp/ipykernel_2568289/2063573927.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [19]:
# Calculate the difference between the best imputation method for each data constellation to the average best imputation method in F1 score
performance_difference['Performance Difference Best to Average'] = performance_difference['Imputed_best'] - performance_difference['Imputed_average']
Average_Difference = performance_difference['Performance Difference Best to Average'].mean()
print("Average Difference in Improvement from best method to average best method for RMSE", Average_Difference)


Average Difference in Improvement from best method to average best method for RMSE -0.002863331727930597


In [20]:
performance_difference.to_csv('performance_difference.csv')
performance_difference.head()

,Imputation_Method_average,Imputed_average,Data_Constellation,Downstream Performance Rank Average,Imputation_Method_best,Imputed_best,Downstream Performance Rank Best,Performance Difference Best to Average
0,Random Forest,0.789810,MAR - 0.01 - 310,3.0,Mean/Mode,0.780147,1.0,-0.009662
1,Random Forest,0.789810,MAR - 0.1 - 310,4.0,Mean/Mode,0.780147,1.0,-0.009662
2,Random Forest,0.789810,MAR - 0.3 - 310,3.0,Mean/Mode,0.780147,1.0,-0.009662
3,Random Forest,0.789810,MAR - 0.5 - 310,4.0,Mean/Mode,0.780147,1.0,-0.009662
4,Random Forest,0.774143,MCAR - 0.01 - 310,3.0,Mean/Mode,0.774143,1.0,0.000000


## Analysis and Ranking based on RMSE Score

In [21]:
# Relative Difference in Percent -> Best Method to Average Best Method

data = downstream_results_rank.copy()
data['Task'] = data['Task'].astype(str)
data['Data_Constellation_full'] = data['Data_Constellation'] + ' - ' + data['Task']

dc_unique = data.Data_Constellation_full.unique()

data_constellations = dc_unique.tolist()
methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
average_best_complete = pd.DataFrame()


for i in data_constellations:
    data_constel = data.loc[data['Data_Constellation_full'] == i]
    best_score = data_constel.loc[data_constel['Downstream Performance Rank'] == 1.0]
    average_best = data_constel.loc[data_constel['Imputation_Method'] == AVERAGE_BEST_IMPUTATION_METHOD]
    best_score_int = best_score.iloc[0]['Imputed']
    average_best_int = average_best.iloc[0]['Imputed']
    calc_result = ((best_score_int - average_best_int)/best_score_int)
    calc_result = abs(calc_result)
    average_best['Performance Difference to Best to Average in Percent'] = calc_result
    average_best_complete = average_best_complete.append(average_best)

average_best_complete

IndexError: single positional indexer is out-of-bounds

In [ ]:
print(downstream_results_rank)

Empty DataFrame
Columns: [result_type, metric, Baseline, Corrupted, Imputed, Imputation_Method, Missing Type, Missing Fraction, Column, Unnamed: 0, Task, name, MajorityClassSize, MinorityClassSize, NumberOfFeatures, NumberOfInstances, NumberOfNumericFeatures, NumberOfCategoricalFeatures, NumberOfClasses, Downstream Performance Rank, Data_Constellation]
Index: []

[0 rows x 21 columns]


In [ ]:
#Difference in Percentage
average_difference = average_best_complete['Performance Difference to Best to Average in Percent'].mean()
print(average_difference, "average difference in Percent (Verschlechterung der durchschnittlich besten Methode relativ zu jeweils besten Methode)")

KeyError: 'Performance Difference to Best to Average in Percent'

## Heatmap to Show Detailled Performance of Each Imputation Method for Each Data Constellation

In [22]:
df_heat = downstream_results_rank.copy()
df_heat.drop(["Missing Type", "Missing Fraction", "Column", "result_type", "metric", "Baseline", "Corrupted", "Unnamed: 0", "Unnamed: 0", "name", "NumberOfClasses", "MajorityClassSize", "MinorityClassSize"], axis=1, inplace=True)
df_heat

,Imputation_Method,Task,Imputed,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Downstream Performance Rank,Data_Constellation
0,Mean/Mode,6,0.725661,17.0,20000.0,16.0,1.0,1.0,MAR - 0.01
1,KNN,6,0.726121,17.0,20000.0,16.0,1.0,2.0,MAR - 0.01
2,Mean/Mode,6,0.725661,17.0,20000.0,16.0,1.0,1.0,MAR - 0.1
3,KNN,6,0.726121,17.0,20000.0,16.0,1.0,2.0,MAR - 0.1
4,Mean/Mode,6,0.725661,17.0,20000.0,16.0,1.0,1.0,MAR - 0.3
...,...,...,...,...,...,...,...,...,...
2061,Mean/Mode,42712,149.536270,13.0,17379.0,9.0,4.0,2.0,MNAR - 0.1
2062,KNN,42712,149.519166,13.0,17379.0,9.0,4.0,1.0,MNAR - 0.3
2063,Mean/Mode,42712,149.536270,13.0,17379.0,9.0,4.0,2.0,MNAR - 0.3
2064,KNN,42712,149.519166,13.0,17379.0,9.0,4.0,1.0,MNAR - 0.5


In [23]:
# Heatmap for total RMSE score for each data constellation for each method

df_heat = df_heat.astype({"Task":"string"})

data_constellations = ['MAR - 0.01', 'MAR - 0.1', 'MAR - 0.3', 'MCAR - 0.5', 'MCAR - 0.01', 'MCAR - 0.1', 'MCAR - 0.3', 'MCAR - 0.5', 'MNAR - 0.01', 'MNAR - 0.1', 'MNAR - 0.3', 'MNAR - 0.5']


for i in data_constellations:
    data_constel = df_heat.loc[df_heat['Data_Constellation'] == i]

    ### uncomment whatever you want to investigate

    ## sort by amount datapoints (ascending)
    data_constel = data_constel.sort_values(by=['NumberOfInstances'])

    ## sort by amount of features (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfFeatures'])

    ## sort by amount of datapoints and features (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfInstances', 'NumberOfFeatures'])

    ## sort by amount of categorical features and datapoints (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfCategoricalFeatures', 'NumberOfInstances'])

    ## sort by amount of numerical features and datapoints (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfNumericFeatures', 'NumberOfInstances'])
    
    Dataset_number = data_constel["Task"]
    Imputation_Method = data_constel["Imputation_Method"]
    Improvement = data_constel["Imputed"]
    

    trace = go.Heatmap(
                   z=Improvement,
                   x=Dataset_number,
                   y=Imputation_Method,
                   type = 'heatmap',
                    autocolorscale= False,
                    colorscale = 'Reds',
                    zmin=0,
                    )
    data = [trace]
    fig = go.Figure(data=data)
    fig.update_layout(
        title=i,
        xaxis_nticks=36)
    fig.show()

In [ ]:

df_heat_dif = downstream_results_rank_heatmap2.copy()
#df_heat_dif

In [ ]:
# Calculate Difference for every Imputation towards average best Imputation Method per Data Constellation
# Calculation for RMSE Differences in Percentage (RMSE alone not comparable over different datasets)

data = downstream_results_rank.copy()
data['Task'] = data['Task'].astype(str)
data['Data_Constellation_full'] = data['Data_Constellation'] + ' - ' + data['Task']

dc_unique = data.Data_Constellation_full.unique()
#print(dc_unique)

data_constellations = dc_unique.tolist()
methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
heatmap_data_difference = pd.DataFrame()


for i in data_constellations:
    data_constel = data.loc[data['Data_Constellation_full'] == i]
    average_best = data_constel.loc[data_constel['Imputation_Method'] == AVERAGE_BEST_IMPUTATION_METHOD]
    dataset_number = best_score.iloc[0]['Task']
    for i in methods:
        if ((data_constel['Imputation_Method'] == i).any()):
            current_score_row = data_constel.loc[data['Imputation_Method'] == i]
            current_score_int = current_score_row.iloc[0]['Imputed']
            average_best_int = average_best.iloc[0]['Imputed']
            calc_result = ((average_best_int - current_score_int)/current_score_int)

            current_score_row['Performance Difference to Average Best in Percent'] = calc_result
            heatmap_data_difference = heatmap_data_difference.append(current_score_row)  
        else:
            print("Imputation Method not here ---------------------")

heatmap_data_difference



Imputation Method not here ---------------------


IndexError: single positional indexer is out-of-bounds

In [ ]:
# Heatmap for RMSE differences in Percentage for each data constellation for each method relative to average best imputation method

heatmap_data_difference = heatmap_data_difference.astype({"Task":"string"})
data_constellations = ['MAR - 0.01', 'MAR - 0.1', 'MAR - 0.3', 'MAR - 0.5', 'MCAR - 0.01', 'MCAR - 0.1', 'MCAR - 0.3', 'MCAR - 0.5', 'MNAR - 0.01', 'MNAR - 0.1', 'MNAR - 0.3', 'MNAR - 0.5']
for i in data_constellations:
    data_constel = heatmap_data_difference.loc[df_heat['Data_Constellation'] == i]

    ### uncomment whatever you want to investigate

    ## sort by amount datapoints (ascending)
    data_constel = data_constel.sort_values(by=['NumberOfInstances'])

    ## sort by amount of features (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfFeatures'])

    ## sort by amount of datapoints and features (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfInstances', 'NumberOfFeatures'])

    ## sort by amount of categorical features and datapoints (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfCategoricalFeatures', 'NumberOfInstances'])

    ## sort by amount of numerical features and datapoints (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfNumericFeatures', 'NumberOfInstances'])
    
    Dataset_number = data_constel["Task"]
    Imputation_Method = data_constel["Imputation_Method"]
    Improvement = data_constel["Performance Difference to Average Best in Percent"]
    

    trace = go.Heatmap(
                   z=Improvement,
                   x=Dataset_number,
                   y=Imputation_Method,
                   type = 'heatmap',
                    autocolorscale= False,
                    colorscale = 'RdBu_r',
                    zmid=0,
                    zmin=(-0.1),
                    zmax=0.1,
                    )
    data = [trace]
    fig = go.Figure(data=data)
    fig.update_layout(
        title=i,
        xaxis_nticks=36)
    fig.show()
    fig.write_image("regression_heatmap_f1_score_improvement_to_avbest%s.pdf" %i)

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'Task' not found in columns."

In [ ]:
#heatmap_data_difference.agg(['min', 'max'])
heatmap_data_difference.to_csv('heatmap_data_difference.csv')
heatmap_data_difference['Performance Difference to Average Best in Percent'].agg(['min', 'max'])

KeyError: 'Performance Difference to Average Best in Percent'

In [ ]:
heatmap_data_difference
heatmap_data_difference.to_csv('regression_imputed_full_info.csv', index=False)

## Improvment Proportions for All Data Constellations and Methods Relative to Average Best Method

In [ ]:
# data preprocessing here
df_quantiles = heatmap_data_difference.copy()
df_quantiles = df_quantiles.drop(df_quantiles[df_quantiles["Imputation_Method"] == AVERAGE_BEST_IMPUTATION_METHOD].index)

df_10 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] > (-0.09))].index)
df_09 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (-0.09)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (-0.07))].index)
df_07 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (-0.07)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (-0.05))].index)
df_05 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (-0.05)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (-0.03))].index)
df_03 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (-0.03)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (-0.01))].index)
df_01 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (-0.01)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (0.01))].index)
df01 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (0.01)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (0.03))].index)
df03 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (0.03)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (0.05))].index)
df05 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (0.05)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (0.07))].index)
df07 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (0.07)) | (df_quantiles["Performance Difference to Average Best in Percent"] > (0.09))].index)
df09 = df_quantiles.drop(df_quantiles[(df_quantiles["Performance Difference to Average Best in Percent"] <= (0.09))].index)

#df_quantiles
#df_quantiles.dtypes

KeyError: 'Imputation_Method'

In [ ]:
len_df_10 = len(df_10.index)
len_df_09 = len(df_09.index)
len_df_07 = len(df_07.index)
len_df_05 = len(df_05.index)
len_df_03 = len(df_03.index)
len_df_01 = len(df_01.index)
len_df01 = len(df01.index)
len_df03 = len(df03.index)
len_df05 = len(df05.index)
len_df07 = len(df07.index)
len_df09 = len(df09.index)

quantile_freq = []
#quantile_freq.append()
quantile_freq.extend((len_df_10, len_df_09, len_df_07, len_df_05, len_df_03, len_df_01, len_df01, len_df03, len_df05, len_df07, len_df09))
print(quantile_freq)


quantiles = []
quantiles.extend(['less than -0.09', '-0.09 to -0.07', '-0.07 to -0.05', '-0.05 to -0.03','-0.03 to -0.01', '-0.01 to 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09'])
print(quantiles)

improvement_quantiles = pd.DataFrame(
    {'Improvement to Average Best in Percent': quantiles,
     'Amount': quantile_freq,
    })


[10, 6, 8, 20, 46, 1009, 12, 2, 0, 0, 1]
['less than -0.09', '-0.09 to -0.07', '-0.07 to -0.05', '-0.05 to -0.03', '-0.03 to -0.01', '-0.01 to 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


In [ ]:
fig = px.bar(improvement_quantiles, x='Improvement to Average Best in Percent', y='Amount')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl.pdf")

In [ ]:
# split barchart stacks into methods

quantile_datasets = [df_10, df_09, df_07, df_05, df_03, df_01, df01, df03, df05, df07, df09]

methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
methods.remove(AVERAGE_BEST_IMPUTATION_METHOD)
print(methods)

forest_freq = []
knn_freq = []
mode_freq = []
dl_freq = []
vae_freq = []
gain_freq = []
#print(quantile_datasets)

for i in methods:
    for j in quantile_datasets:
        df_temp = j.copy()
        df_temp = df_temp[df_temp['Imputation_Method'].str.contains(i)]
        df_temp_len = len(df_temp.index)
        if (i == 'Random Forest'):
            forest_freq.append(df_temp_len)
        elif (i == 'KNN'):
            knn_freq.append(df_temp_len)                                       
        elif (i == 'Mean/Mode'):
            mode_freq.append(df_temp_len)                                                 
        elif (i == 'Discriminative DL'):
            dl_freq.append(df_temp_len)                                       
        elif (i == 'VAE'):
            vae_freq.append(df_temp_len)                                         
        elif (i == 'GAIN'):
            gain_freq.append(df_temp_len)                                          
                                       
print(forest_freq)
print(knn_freq)
print(mode_freq)
print(dl_freq)
print(vae_freq)
print(gain_freq)

['KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
[]
[0, 0, 0, 0, 3, 223, 2, 0, 0, 0, 0]
[0, 1, 3, 7, 9, 205, 2, 1, 0, 0, 0]
[0, 0, 0, 0, 3, 216, 3, 0, 0, 0, 0]
[3, 3, 2, 4, 10, 203, 1, 1, 0, 0, 1]
[7, 2, 3, 9, 21, 162, 4, 0, 0, 0, 0]


In [ ]:
quantiles = ['less than -0.09', '-0.09 to -0.07', '-0.07 to -0.05', '-0.05 to -0.03','-0.03 to -0.01', '-0.01 to 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']

fig = go.Figure(data=[
    go.Bar(name='Random Forest', x=quantiles, y=forest_freq),
    go.Bar(name='KNN', x=quantiles, y=knn_freq),
    go.Bar(name='Mean/Mode', x=quantiles, y=mode_freq),
    go.Bar(name='Discriminative DL', x=quantiles, y=dl_freq),
    go.Bar(name='VAE', x=quantiles, y=vae_freq),
    go.Bar(name='GAIN', x=quantiles, y=gain_freq)
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_per_method.pdf")

In [ ]:
# split barchart stacks into methods

quantile_datasets = [df_10, df_09, df_07, df_05, df_03, df_01, df01, df03, df05, df07, df09]

fractions = ['0.01', '0.1', '0.3', '0.5']
#print(fractions)
#print(df_10)

freq_001 = []
freq_01 = []
freq_03 = []
freq_05 = []
#print(quantile_datasets)

for i in fractions:
    for j in quantile_datasets:
        df_temp = j.copy()
        df_temp = df_temp[df_temp['Missing Fraction'].str.contains(i)]
        df_temp_len = len(df_temp.index)
        if (i == '0.01'):
            freq_001.append(df_temp_len)
        elif (i == '0.1'):
            freq_01.append(df_temp_len)                                       
        elif (i == '0.3'):
            freq_03.append(df_temp_len)                                                 
        elif (i == '0.5'):
            freq_05.append(df_temp_len)                                       
                                        
                                       
print(freq_001)
print(freq_01)
print(freq_03)
print(freq_05)

[0, 0, 0, 0, 0, 267, 0, 0, 0, 0, 0]
[2, 0, 1, 1, 17, 259, 0, 0, 0, 0, 0]
[3, 3, 5, 7, 12, 248, 5, 0, 0, 0, 1]
[5, 3, 2, 12, 17, 235, 7, 2, 0, 0, 0]


In [ ]:
quantiles = ['less than -0.09', '-0.09 to -0.07', '-0.07 to -0.05', '-0.05 to -0.03','-0.03 to -0.01', '-0.01 to 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


fig = go.Figure(data=[
    go.Bar(name='1% Missing Data', x=quantiles, y=freq_001, marker_color='#FD3216'),
    go.Bar(name='10% Missing Data', x=quantiles, y=freq_01, marker_color='#00FE35'),
    go.Bar(name='30% Missing Data', x=quantiles, y=freq_03, marker_color='#511CFB'),
    go.Bar(name='50% Missing Data', x=quantiles, y=freq_05, marker_color='#FF7F0E'),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_per_frac.pdf")

In [ ]:
# split barchart stacks into methods

quantile_datasets = [df_10, df_09, df_07, df_05, df_03, df_01, df01, df03, df05, df07, df09]

fractions = ['MCAR', 'MAR', 'MNAR']
print(fractions)
#print(df_10)

freq_001 = []
freq_01 = []
freq_03 = []
#print(quantile_datasets)

for i in fractions:
    for j in quantile_datasets:
        df_temp = j.copy()
        df_temp = df_temp[df_temp['Missing Type'].str.contains(i)]
        df_temp_len = len(df_temp.index)
        if (i == 'MCAR'):
            freq_001.append(df_temp_len)
        elif (i == 'MAR'):
            freq_01.append(df_temp_len)                                       
        elif (i == 'MNAR'):
            freq_03.append(df_temp_len)                                                 

                                       
print(freq_001)
print(freq_01)
print(freq_03)

['MCAR', 'MAR', 'MNAR']
[3, 4, 1, 8, 13, 341, 0, 0, 0, 0, 0]
[4, 2, 1, 6, 19, 336, 3, 0, 0, 0, 0]
[3, 0, 6, 6, 14, 332, 9, 2, 0, 0, 1]


In [ ]:
quantiles = ['less than -0.09', '-0.09 to -0.07', '-0.07 to -0.05', '-0.05 to -0.03','-0.03 to -0.01', '-0.01 to 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


fig = go.Figure(data=[
    go.Bar(name='MCAR', x=quantiles, y=freq_001, marker_color='#222A2A'),
    go.Bar(name='MAR', x=quantiles, y=freq_01, marker_color='#B68100'),
    go.Bar(name='MNAR', x=quantiles, y=freq_03, marker_color='#750D86'),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_per_patt.pdf")

## Improvment Proportions for the Best Imputation Method per Data Constellation Relative to Average Best Method

In [ ]:
improv_to_av_bar = heatmap_data_difference.copy()

improv_to_av_bar = improv_to_av_bar.drop(improv_to_av_bar[improv_to_av_bar["Downstream Performance Rank"] != 1.0].index)

df_01 = improv_to_av_bar.drop(improv_to_av_bar[(improv_to_av_bar["Performance Difference to Average Best in Percent"] <= (-0.01)) | (improv_to_av_bar["Performance Difference to Average Best in Percent"] > (0.01))].index)
df01 = improv_to_av_bar.drop(improv_to_av_bar[(improv_to_av_bar["Performance Difference to Average Best in Percent"] <= (0.01)) | (improv_to_av_bar["Performance Difference to Average Best in Percent"] > (0.03))].index)
df03 = improv_to_av_bar.drop(improv_to_av_bar[(improv_to_av_bar["Performance Difference to Average Best in Percent"] <= (0.03)) | (improv_to_av_bar["Performance Difference to Average Best in Percent"] > (0.05))].index)
df05 = improv_to_av_bar.drop(improv_to_av_bar[(improv_to_av_bar["Performance Difference to Average Best in Percent"] <= (0.05)) | (improv_to_av_bar["Performance Difference to Average Best in Percent"] > (0.07))].index)
df07 = improv_to_av_bar.drop(improv_to_av_bar[(improv_to_av_bar["Performance Difference to Average Best in Percent"] <= (0.07)) | (improv_to_av_bar["Performance Difference to Average Best in Percent"] > (0.09))].index)
df09 = improv_to_av_bar.drop(improv_to_av_bar[(improv_to_av_bar["Performance Difference to Average Best in Percent"] <= (0.09))].index)

improv_to_av_bar

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best in Percent
0,GAIN,189,MAR,0.01,theta8,downstream_performance_mean,Regression Tasks,0.204873,0.0,0.204926,...,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.01,MAR - 0.01 - 189,0.000356
6,Discriminative DL,189,MAR,0.1,theta8,downstream_performance_mean,Regression Tasks,0.204813,0.0,0.204953,...,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.1,MAR - 0.1 - 189,0.000270
12,Mean/Mode,189,MAR,0.3,theta8,downstream_performance_mean,Regression Tasks,0.204820,0.0,0.205069,...,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.3,MAR - 0.3 - 189,0.002178
18,Discriminative DL,189,MAR,0.5,theta8,downstream_performance_mean,Regression Tasks,0.205841,0.0,0.205878,...,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.5,MAR - 0.5 - 189,0.000489
24,Discriminative DL,189,MCAR,0.01,theta8,downstream_performance_mean,Regression Tasks,0.205083,0.0,0.204972,...,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MCAR - 0.01,MCAR - 0.01 - 189,0.000514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,Random Forest,42712,MCAR,0.5,humidity,downstream_performance_mean,Regression Tasks,148.991778,0.0,148.880924,...,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MCAR - 0.5,MCAR - 0.5 - 42712,0.000000
1318,Discriminative DL,42712,MNAR,0.01,humidity,downstream_performance_mean,Regression Tasks,149.507994,0.0,149.475688,...,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.01,MNAR - 0.01 - 42712,0.000490
1324,Discriminative DL,42712,MNAR,0.1,humidity,downstream_performance_mean,Regression Tasks,149.523046,0.0,149.372273,...,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.1,MNAR - 0.1 - 42712,0.002744
1330,Discriminative DL,42712,MNAR,0.3,humidity,downstream_performance_mean,Regression Tasks,149.431371,0.0,149.346814,...,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.3,MNAR - 0.3 - 42712,0.004428


In [ ]:
#improv_to_av_bar.agg(['min', 'max'])

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best in Percent
min,Discriminative DL,1193,MAR,0.01,Material,downstream_performance_mean,Regression Tasks,0.001417,0.0,0.001418,...,NaN,6.0,4477.0,3.0,0.0,NaN,1.0,MAR - 0.01,MAR - 0.01 - 1193,0.000000
max,VAE,42712,MNAR,0.5,x6,downstream_performance_mean,Regression Tasks,72725.060283,0.0,72725.053954,...,NaN,22.0,89640.0,22.0,7.0,NaN,1.0,MNAR - 0.5,MNAR - 0.5 - 42712,0.160131


In [ ]:
len_df_01 = len(df_01.index)
len_df01 = len(df01.index)
len_df03 = len(df03.index)
len_df05 = len(df05.index)
len_df07 = len(df07.index)
len_df09 = len(df09.index)

quantile_freq = []
quantile_freq.extend((len_df_01, len_df01, len_df03, len_df05, len_df07, len_df09))
print(quantile_freq)


quantiles = []
quantiles.extend(['less than 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09'])
print(quantiles)

improvement_quantiles = pd.DataFrame(
    {'Improvement to Average Best in Percent': quantiles,
     'Amount': quantile_freq,
    })

fig = px.bar(improvement_quantiles, x='Improvement to Average Best in Percent', y='Amount')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_only_best.pdf")

[218, 7, 2, 0, 0, 1]
['less than 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


In [ ]:
# split barchart stacks into methods

quantile_datasets = [df_01, df01, df03, df05, df07, df09]

methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
methods.remove(AVERAGE_BEST_IMPUTATION_METHOD)
print(methods)

forest_freq = []
knn_freq = []
mode_freq = []
dl_freq = []
vae_freq = []
gain_freq = []
#print(quantile_datasets)

for i in methods:
    for j in quantile_datasets:
        df_temp = j.copy()
        df_temp = df_temp[df_temp['Imputation_Method'].str.contains(i)]
        df_temp_len = len(df_temp.index)
        if (i == 'Random Forest'):
            forest_freq.append(df_temp_len)
        elif (i == 'KNN'):
            knn_freq.append(df_temp_len)                                       
        elif (i == 'Mean/Mode'):
            mode_freq.append(df_temp_len)                                                 
        elif (i == 'Discriminative DL'):
            dl_freq.append(df_temp_len)                                       
        elif (i == 'VAE'):
            vae_freq.append(df_temp_len)                                         
        elif (i == 'GAIN'):
            gain_freq.append(df_temp_len)                                          
                                       
print(forest_freq)
print(knn_freq)
print(mode_freq)
print(dl_freq)
print(vae_freq)
print(gain_freq)

['KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
[]
[23, 1, 0, 0, 0, 0]
[47, 2, 1, 0, 0, 0]
[35, 2, 0, 0, 0, 0]
[29, 0, 1, 0, 0, 1]
[36, 2, 0, 0, 0, 0]


In [ ]:
quantiles = ['less than 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


fig = go.Figure(data=[
    go.Bar(name='Random Forest', x=quantiles, y=forest_freq),
    go.Bar(name='KNN', x=quantiles, y=knn_freq),
    go.Bar(name='Mean/Mode', x=quantiles, y=mode_freq),
    go.Bar(name='Discriminative DL', x=quantiles, y=dl_freq),
    go.Bar(name='VAE', x=quantiles, y=vae_freq),
    go.Bar(name='GAIN', x=quantiles, y=gain_freq)
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_only_best_per_method.pdf")

In [ ]:
# split barchart stacks into missingness fractions

quantile_datasets = [df_01, df01, df03, df05, df07, df09]

fractions = ['0.01', '0.1', '0.3', '0.5']
print(fractions)


freq_001 = []
freq_01 = []
freq_03 = []
freq_05 = []
#print(quantile_datasets)

for i in fractions:
    for j in quantile_datasets:
        df_temp = j.copy()
        df_temp = df_temp[df_temp['Missing Fraction'].str.contains(i)]
        df_temp_len = len(df_temp.index)
        if (i == '0.01'):
            freq_001.append(df_temp_len)
        elif (i == '0.1'):
            freq_01.append(df_temp_len)                                       
        elif (i == '0.3'):
            freq_03.append(df_temp_len)                                                 
        elif (i == '0.5'):
            freq_05.append(df_temp_len)                                       
                                        
                                       
print(freq_001)
print(freq_01)
print(freq_03)
print(freq_05)

['0.01', '0.1', '0.3', '0.5']
[57, 0, 0, 0, 0, 0]
[57, 0, 0, 0, 0, 0]
[53, 3, 0, 0, 0, 1]
[51, 4, 2, 0, 0, 0]


In [ ]:
quantiles = ['less than 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


fig = go.Figure(data=[
    go.Bar(name='1% Missing Data', x=quantiles, y=freq_001, marker_color='#FD3216'),
    go.Bar(name='10% Missing Data', x=quantiles, y=freq_01, marker_color='#00FE35'),
    go.Bar(name='30% Missing Data', x=quantiles, y=freq_03, marker_color='#511CFB'),
    go.Bar(name='50% Missing Data', x=quantiles, y=freq_05, marker_color='#FF7F0E'),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_only_best_per_frac.pdf")

In [ ]:
# split barchart stacks into missingness fractions

quantile_datasets = [df_01, df01, df03, df05, df07, df09]

fractions = ['MCAR', 'MAR', 'MNAR']
print(fractions)


freq_001 = []
freq_01 = []
freq_03 = []
#print(quantile_datasets)

for i in fractions:
    for j in quantile_datasets:
        df_temp = j.copy()
        df_temp = df_temp[df_temp['Missing Type'].str.contains(i)]
        df_temp_len = len(df_temp.index)
        if (i == 'MCAR'):
            freq_001.append(df_temp_len)
        elif (i == 'MAR'):
            freq_01.append(df_temp_len)                                       
        elif (i == 'MNAR'):
            freq_03.append(df_temp_len)                                                 

                                       
print(freq_001)
print(freq_01)
print(freq_03)

['MCAR', 'MAR', 'MNAR']
[76, 0, 0, 0, 0, 0]
[73, 3, 0, 0, 0, 0]
[69, 4, 2, 0, 0, 1]


In [ ]:
quantiles = ['less than 0.01', '0.01 to 0.03', '0.03 to 0.05', '0.05 to 0.07', '0.07 to 0.09', 'more than 0.09']


fig = go.Figure(data=[
    go.Bar(name='MCAR', x=quantiles, y=freq_001, marker_color='#222A2A'),
    go.Bar(name='MAR', x=quantiles, y=freq_01, marker_color='#B68100'),
    go.Bar(name='MNAR', x=quantiles, y=freq_03, marker_color='#750D86'),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()
fig.write_image("improv_rel_to_av_all_DC_no_av_incl_only_best_per_patt.pdf")

## Extract datasets for Automated Imputation Method Selection -> not used for thesis

To Do: Explore the possibility, that the average best method replaces the best method for a data constellation, if the improvement gain for the best method is below 1%

### Potential Features:
Missingess Pattern (Missing Type)  
Missing Fraction (Missing Fraction)  
Datapoints (NumberOfInstances)  
Features in total (NumberOfFeatures)  
Numeric Features (NumberOfNumericFeatures)  
Categorical Features (NumberOfCategoricalFeatures)  
Downstream Task Type -> Classification/Regression (metric)
  
    
      
Label: Best Imputation Method (Imputation_Method)

In [ ]:
# Use dataset with only the best method for each data constellation
rank_1_backup.to_csv('rank_1_backup.csv')
rank_1_backup

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation
0,GAIN,189,MAR,0.01,theta8,downstream_performance_mean,Regression Tasks,0.204873,0.0,0.204926,...,kin8nm,NaN,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.01
6,Discriminative DL,189,MAR,0.1,theta8,downstream_performance_mean,Regression Tasks,0.204813,0.0,0.204953,...,kin8nm,NaN,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.1
12,Mean/Mode,189,MAR,0.3,theta8,downstream_performance_mean,Regression Tasks,0.204820,0.0,0.205069,...,kin8nm,NaN,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.3
18,Discriminative DL,189,MAR,0.5,theta8,downstream_performance_mean,Regression Tasks,0.205841,0.0,0.205878,...,kin8nm,NaN,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MAR - 0.5
24,Discriminative DL,189,MCAR,0.01,theta8,downstream_performance_mean,Regression Tasks,0.205083,0.0,0.204972,...,kin8nm,NaN,NaN,9.0,8192.0,9.0,0.0,NaN,1.0,MCAR - 0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,Random Forest,42712,MCAR,0.5,humidity,downstream_performance_mean,Regression Tasks,148.991778,0.0,148.880924,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MCAR - 0.5
1318,Discriminative DL,42712,MNAR,0.01,humidity,downstream_performance_mean,Regression Tasks,149.507994,0.0,149.475688,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.01
1324,Discriminative DL,42712,MNAR,0.1,humidity,downstream_performance_mean,Regression Tasks,149.523046,0.0,149.372273,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.1
1330,Discriminative DL,42712,MNAR,0.3,humidity,downstream_performance_mean,Regression Tasks,149.431371,0.0,149.346814,...,Bike_Sharing_Demand,NaN,NaN,13.0,17379.0,9.0,4.0,NaN,1.0,MNAR - 0.3


In [ ]:
# Dataset for Training 
properties_train_dataset_8 = rank_1_backup.copy()
properties_train_dataset_8 = properties_train_dataset_8[['Imputation_Method','Missing Type','Missing Fraction',
                                                         'NumberOfInstances','NumberOfFeatures','NumberOfNumericFeatures',
                                                         'NumberOfCategoricalFeatures','metric']]

properties_train_dataset_8


,Imputation_Method,Missing Type,Missing Fraction,NumberOfInstances,NumberOfFeatures,NumberOfNumericFeatures,NumberOfCategoricalFeatures,metric
0,GAIN,MAR,0.01,8192.0,9.0,9.0,0.0,Regression Tasks
6,Discriminative DL,MAR,0.1,8192.0,9.0,9.0,0.0,Regression Tasks
12,Mean/Mode,MAR,0.3,8192.0,9.0,9.0,0.0,Regression Tasks
18,Discriminative DL,MAR,0.5,8192.0,9.0,9.0,0.0,Regression Tasks
24,Discriminative DL,MCAR,0.01,8192.0,9.0,9.0,0.0,Regression Tasks
...,...,...,...,...,...,...,...,...
1312,Random Forest,MCAR,0.5,17379.0,13.0,9.0,4.0,Regression Tasks
1318,Discriminative DL,MNAR,0.01,17379.0,13.0,9.0,4.0,Regression Tasks
1324,Discriminative DL,MNAR,0.1,17379.0,13.0,9.0,4.0,Regression Tasks
1330,Discriminative DL,MNAR,0.3,17379.0,13.0,9.0,4.0,Regression Tasks


In [ ]:
# Dataset for Training 
properties_train_dataset_7 = rank_1_backup.copy()
properties_train_dataset_7 = properties_train_dataset_7[['Imputation_Method','Missing Type','Missing Fraction',
                                                         'NumberOfInstances','NumberOfFeatures','NumberOfNumericFeatures',
                                                         'NumberOfCategoricalFeatures']]

properties_train_dataset_7
properties_train_dataset_7.to_csv('properties_train_dataset_7.csv', index=False)